## HCCF

In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.init import xavier_normal_, xavier_uniform_
import argparse
import numpy as np
import time
import sys
from os.path import abspath

from util.sampler import  next_batch_pairwise
from util.conf import OptionConf
import torch
import torch.nn as nn 
import torch.nn.functional as F
from scipy.sparse import coo_matrix
from util.loss_torch import bpr_loss, l2_reg_loss, EmbLoss, contrastLoss
from util.init import *
from base.torch_interface import TorchGraphInterface
import os
import numpy as np 
import time 
from torch.optim.lr_scheduler import ReduceLROnPlateau
from itertools import product

from data.loader import FileIO
from util.conf import ModelConf
from base.recommender import Recommender
from data.ui_graph import Interaction
# from data.knowledge import Knowledge
from util.algorithm import find_k_largest
from time import strftime, localtime
from data.loader import FileIO
from util.evaluation import ranking_evaluation

import pandas as pd
from datetime import datetime
import csv

/home/aiotlab3/anaconda3/envs/hungvv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
class GraphRecommender(Recommender):
    def __init__(self, conf, training_set, test_set, knowledge_set, **kwargs):
        super(GraphRecommender, self).__init__(conf, training_set, test_set, knowledge_set, **kwargs)
        self.data = Interaction(conf, training_set, test_set)
        self.bestPerformance = []
        top = self.ranking['-topN'].split(',')
        self.topN = [int(num) for num in top]
        self.max_N = max(self.topN)

    def print_model_info(self):
        super(GraphRecommender, self).print_model_info()
        # # print dataset statistics
        print('Training Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.training_size()))
        print('Test Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.test_size()))
        print('=' * 80)

    def build(self):
        pass

    def train(self):
        pass

    def predict(self, u):
        pass

    def test(self, user_emb, item_emb):
        def process_bar(num, total):
            rate = float(num) / total
            ratenum = int(50 * rate)
            r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
            sys.stdout.write(r)
            sys.stdout.flush()

        # predict
        rec_list = {}
        user_count = len(self.data.test_set)
        for i, user in enumerate(self.data.test_set):
            # s_find_candidates = time.time()
            
            
            # candidates = predict(user)
            user_id  = self.data.get_user_id(user)
            score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
            candidates = score.cpu().numpy()
            
            # e_find_candidates = time.time()
            # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
            # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
            rated_list, li = self.data.user_rated(user)
            for item in rated_list:
                candidates[self.data.item[item]] = -10e8
            
            # s_find_k_largest = time.time()
            ids, scores = find_k_largest(self.max_N, candidates)
            # e_find_k_largest = time.time()
            # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
            item_names = [self.data.id2item[iid] for iid in ids]
            rec_list[user] = list(zip(item_names, scores))
            if i % 1000 == 0:
                process_bar(i, user_count)
        process_bar(user_count, user_count)
        print('')
        return rec_list

    def evaluate(self, rec_list):
        self.recOutput.append('userId: recommendations in (itemId, ranking score) pairs, * means the item is hit.\n')
        for user in self.data.test_set:
            line = str(user) + ':'
            for item in rec_list[user]:
                line += ' (' + str(item[0]) + ',' + str(item[1]) + ')'
                if item[0] in self.data.test_set[user]:
                    line += '*'
            line += '\n'
            self.recOutput.append(line)
        current_time = strftime("%Y-%m-%d %H-%M-%S", localtime(time.time()))
        # output prediction result
        out_dir = self.output['-dir']
        file_name = self.config['model.name'] + '@' + current_time + '-top-' + str(self.max_N) + 'items' + '.txt'
        FileIO.write_file(out_dir, file_name, self.recOutput)
        print('The result has been output to ', abspath(out_dir), '.')
        file_name = self.config['model.name'] + '@' + current_time + '-performance' + '.txt'
        self.result = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        self.model_log.add('###Evaluation Results###')
        self.model_log.add(self.result)
        FileIO.write_file(out_dir, file_name, self.result)
        print('The result of %s:\n%s' % (self.model_name, ''.join(self.result)))

    def fast_evaluation(self, epoch, user_embed, item_embed, kwargs=None):
        print('Evaluating the model...')
        s_test = time.time()
        rec_list = self.test(user_embed, item_embed)
        e_test = time.time() 
        print("Test time: %f s" % (e_test - s_test))
        
        s_measure = time.time()
        measure = ranking_evaluation(self.data.test_set, rec_list, [self.max_N])
        e_measure = time.time()
        print("Measure time: %f s" % (e_measure - s_measure))
        
        if len(self.bestPerformance) > 0:
            count = 0
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            for k in self.bestPerformance[1]:
                if self.bestPerformance[1][k] > performance[k]:
                    count += 1
                else:
                    count -= 1
            if count < 0:
                self.bestPerformance[1] = performance
                self.bestPerformance[0] = epoch + 1
                try:
                    self.save(kwargs)
                except:
                    self.save()
        else:
            self.bestPerformance.append(epoch + 1)
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            self.bestPerformance.append(performance)
            try:
                self.save(kwargs)
            except:
                self.save()
        print('-' * 120)
        print('Real-Time Ranking Performance ' + ' (Top-' + str(self.max_N) + ' Item Recommendation)')
        measure = [m.strip() for m in measure[1:]]
        print('*Current Performance*')
        print('Epoch:', str(epoch + 1) + ',', '  |  '.join(measure))
        bp = ''
        # for k in self.bestPerformance[1]:
        #     bp+=k+':'+str(self.bestPerformance[1][k])+' | '
        bp += 'Hit Ratio' + ':' + str(self.bestPerformance[1]['Hit Ratio']) + '  |  '
        bp += 'Precision' + ':' + str(self.bestPerformance[1]['Precision']) + '  |  '
        bp += 'Recall' + ':' + str(self.bestPerformance[1]['Recall']) + '  |  '
        # bp += 'F1' + ':' + str(self.bestPerformance[1]['F1']) + ' | '
        bp += 'NDCG' + ':' + str(self.bestPerformance[1]['NDCG'])
        print('*Best Performance* ')
        print('Epoch:fast_evaluation', str(self.bestPerformance[0]) + ',', bp)
        print('-' * 120)    
        
             
        performance_ep = {
            'Epoch': epoch,
            'Hit Ratio': str(self.bestPerformance[1]['Hit Ratio']),
            'Precision': str(self.bestPerformance[1]['Precision']),
            'Recall': str(self.bestPerformance[1]['Recall']),
            'NDCG': str(self.bestPerformance[1]['NDCG']),
            'Measure Time': e_measure - s_measure
        }
        
        return measure, performance_ep
        
    def save_model(self, model, hyper_params):
        hp = list(hyper_params.values())
        hyper_params = '-p:' + str(hp[0]) + '-drop:' + str(hp[1]) + '-layers:' + str(hp[2]) + '-input_dim:' + str(hp[3]) + '-hyper_dim:' + str(hp[4]) + '-hyperedge_num:' + str(hp[5]) + 'lr:' + str(hp[6]) + 'reg:' + str(hp[7])
        # save model 
        current_time = strftime("%Y-%m-%d", localtime(time.time()))
        out_dir = 'results/'+ str(hp[9]) + '/' + self.config['model.name'] + '/' +self.config['model.name'] + hyper_params
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        file_name =  self.config['model.name'] + '@' + current_time + '-weight' + '.pth'
        weight_file = out_dir + '/' + file_name 
        torch.save(model.state_dict(), weight_file)

    def save_loss(self, train_losses, rec_losses, reg_losses, hyper_params):
        hp = list(hyper_params.values())
        hyper_params = '-p:' + str(hp[0]) + '-drop:' + str(hp[1]) + '-layers:' + str(hp[2]) + '-input_dim:' + str(hp[3]) + '-hyper_dim:' + str(hp[4]) + '-hyperedge_num:' + str(hp[5]) + 'lr:' + str(hp[6]) + 'reg:' + str(hp[7])
        out_dir = 'results/' + str(hp[9]) + '/' + self.config['model.name'] + '/' +self.config['model.name'] + hyper_params
    
        df_train_loss = pd.DataFrame(train_losses, columns = ['ep', 'loss'])
        df_rec_loss = pd.DataFrame(rec_losses, columns = ['ep', 'loss'])
        df_reg_loss = pd.DataFrame(reg_losses, columns = ['ep', 'loss'])
        
        df_train_loss.to_csv(out_dir + '/train_loss.csv')
        df_rec_loss.to_csv(out_dir + '/rec_loss.csv')
        df_reg_loss.to_csv(out_dir + '/reg_loss.csv')

    def save_perfomance_training(self, log_train, hyper_params):
        hp = list(hyper_params.values())
        hyper_params = '-p:' + str(hp[0]) + '-drop:' + str(hp[1]) + '-layers:' + str(hp[2]) + '-input_dim:' + str(hp[3]) + '-hyper_dim:' + str(hp[4]) + '-hyperedge_num:' + str(hp[5]) + 'lr:' + str(hp[6]) + 'reg:' + str(hp[7])
        out_dir = 'results/' + str(hp[9]) + '/' + self.config['model.name'] + '/' +self.config['model.name'] + hyper_params
        
        df_train_log = pd.DataFrame(log_train)
        df_train_log.to_csv(out_dir + '/train_performance.csv')
    
    def save_performance_test(self, log_test, hyper_params):
        hp = list(hyper_params.values())
        hyper_params = '-p:' + str(hp[0]) + '-drop:' + str(hp[1]) + '-layers:' + str(hp[2]) + '-input_dim:' + str(hp[3]) + '-hyper_dim:' + str(hp[4]) + '-hyperedge_num:' + str(hp[5]) + 'lr:' + str(hp[6]) + 'reg:' + str(hp[7])
        out_dir = 'results/' + str(hp[9]) + '/' + self.config['model.name'] + '/' +self.config['model.name'] + hyper_params
        
        df_test_log = pd.DataFrame(log_test)
        df_test_log.to_csv(out_dir + '/test_performance.csv')

In [5]:
class HCCFEncoder(nn.Module):
    def __init__(self, data, emb_size, n_layers, hyperedge_num, leaky, drop_rate):
        super(HCCFEncoder, self).__init__()
        self.data = data
        self.leaky = leaky
        self.n_layers = n_layers
        self.drop_rate = drop_rate
        self.gcnlayer = GCNLayer(self.leaky)
        self.hgnnlayer = HGNNLayer(self.leaky)
        self.norm_adj = data.norm_adj
        self.latent_size = emb_size
        self.n_edges = hyperedge_num
        self.sparse_norm_adj = TorchGraphInterface.convert_sparse_mat_to_tensor(self.norm_adj).to(device)
        self.embedding_dict = self._init_model()
        self.drop_out = nn.Dropout(self.drop_rate)
        self.edgeDropper = SpAdjDropEdge()

    def _init_model(self):
        initializer = nn.init.xavier_uniform_
        embedding_dict = nn.ParameterDict({
            'user_emb': nn.Parameter(initializer(torch.empty(self.data.n_users, self.latent_size)).to(device)),
            'item_emb': nn.Parameter(initializer(torch.empty(self.data.n_items, self.latent_size)).to(device)),
            'user_w': nn.Parameter(initializer(torch.empty(self.latent_size, self.n_edges)).to(device)),
            'item_w': nn.Parameter(initializer(torch.empty(self.latent_size, self.n_edges)).to(device))
        })

#         embedding_dict = nn.ParameterDict({
#             'user_emb': nn.Parameter(torch.empty(self.data.n_users, self.latent_size).to(device)),
#             'item_emb': nn.Parameter(torch.empty(self.data.n_items, self.latent_size).to(device)),
#             'user_w': nn.Parameter(torch.empty(self.latent_size, self.n_edges).to(device)),
#             'item_w': nn.Parameter(torch.empty(self.latent_size, self.n_edges).to(device))
#         })
        
        return embedding_dict
        
    def forward(self):
        embeddings = torch.cat([self.embedding_dict['user_emb'], self.embedding_dict['item_emb']], 0)
        hidden = [embeddings]
        gcn_hidden = []
        hgnn_hidden = []
#         hyper_uu = torch.mm(self.embedding_dict['user_emb'], self.embedding_dict['user_w'])
#         hyper_ii = torch.mm(self.embedding_dict['item_emb'], self.embedding_dict['item_w'])
        
        hyper_uu = self.embedding_dict['user_emb'] @ self.embedding_dict['user_w']
        hyper_ii = self.embedding_dict['item_emb'] @self.embedding_dict['item_w']
        
        for i in range(self.n_layers):
            gcn_emb = self.gcnlayer(self.edgeDropper(self.sparse_norm_adj, 0.5), hidden[-1]) 
            hyper_uemb = self.hgnnlayer(self.drop_out(hyper_uu), hidden[-1][:self.data.n_users])
            hyper_iemb = self.hgnnlayer(self.drop_out(hyper_ii), hidden[-1][self.data.n_users:])
            gcn_hidden += [gcn_emb]
            hgnn_hidden += [torch.cat([hyper_uemb, hyper_iemb], 0)]
            hidden += [gcn_emb + hgnn_hidden[-1]]
            
        embeddings = sum(hidden)
        
        user_emb = embeddings[:self.data.n_users]
        item_emb = embeddings[self.data.n_users:]
        
        return user_emb, item_emb, gcn_hidden, hgnn_hidden
        
class GCNLayer(nn.Module):
    def __init__(self, leaky):
        super(GCNLayer, self).__init__()
        self.act = nn.LeakyReLU(negative_slope=leaky)
        
    def forward(self, adj, embeds):
        return (torch.sparse.mm(adj, embeds))
    
class HGNNLayer(nn.Module):
    def __init__(self, leaky):
        super(HGNNLayer, self).__init__()
        self.act = nn.LeakyReLU(negative_slope=leaky)
    
    def forward(self, adj, embeds):
        #edge_embeds = self.act(torch.mm(adj.T, embeds))
        #hyper_embeds = self.act(torch.mm(adj, edge_embeds))
        edge_embeds = torch.mm(adj.T, embeds)
        hyper_embeds = torch.mm(adj, edge_embeds)
        return hyper_embeds
    
class SpAdjDropEdge(nn.Module):
	def __init__(self):
		super(SpAdjDropEdge, self).__init__()

	def forward(self, adj, keepRate):
		if keepRate == 1.0:
			return adj
		vals = adj._values()
		idxs = adj._indices()
		edgeNum = vals.size()
		mask = ((torch.rand(edgeNum) + keepRate).floor()).type(torch.bool)
		newVals = vals[mask] / keepRate
		newIdxs = idxs[:, mask]
		return torch.sparse.FloatTensor(newIdxs, newVals, adj.shape)
    

In [6]:
def contrastLoss(embeds1, embeds2, nodes, temp):
	embeds1 = F.normalize(embeds1 + 1e-8, p=2)
	embeds2 = F.normalize(embeds2 + 1e-8, p=2)
	pckEmbeds1 = embeds1
	pckEmbeds2 = embeds2
	nume = torch.exp(torch.sum(pckEmbeds1 * pckEmbeds2, dim=-1) / temp)
	deno = torch.exp(pckEmbeds1 @ pckEmbeds2.T / temp).sum(-1) + 1e-8
	return -torch.log(nume / deno).mean()

In [7]:
def calcLosses(ancs, poss, negs, gcnEmbedsLst, hyperEmbedsLst):
    sslLoss = 0
    for i in range(args['n_layers']):
        embeds1 = gcnEmbedsLst[i].detach()
        embeds2 = hyperEmbedsLst[i]
        sslLoss += contrastLoss(embeds1[:rec.data.n_users], embeds2[:rec.data.n_users], torch.unique(ancs.long()), 1) + contrastLoss(embeds1[rec.data.n_users:], embeds2[rec.data.n_users:], torch.unique(poss), 1)
    return sslLoss

In [8]:
def predict(u, rec, user_emb, item_emb):
    user_id  = rec.data.get_user_id(u)
    score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
    return score.cpu().numpy()  

## Train

In [9]:
def train(hyper_params):
    performance_dict = {'Epoch':[], 
                        'Hit Ratio': [], 
                        'Precision':[], 
                        'Recall': [],
                        'NDCG': [], 
                        'Measure Time': []}
    total_train_losses = []
    total_rec_losses = []
    total_reg_losses = []
    
    for ep in range(maxEpoch):
        train_losses = []
        rec_losses = []
        reg_losses = []
        
        for n, batch in enumerate(next_batch_pairwise(rec.data, batchSize)):
            user_idx, pos_idx, neg_idx = batch
            train_model.train()
            user_emb, item_emb, gcnEmbedsLst, hyperEmbedsLst = train_model()

            anchor_emb = user_emb[user_idx]
            pos_emb = item_emb[pos_idx]
            neg_emb = item_emb[neg_idx]
            
            loss_rec = bpr_loss(anchor_emb, pos_emb, neg_emb)
            loss_reg = calcLosses(user_idx, pos_idx, neg_idx, gcnEmbedsLst, hyperEmbedsLst)
    #         print(f"Loss rec: {loss_rec}; Loss reg: {loss_reg}")
            
            batch_loss = loss_rec + loss_reg 
            #total_train_losses.append([loss_rec.item(), loss_reg.item(), batch_loss.item()])
            train_losses.append(batch_loss.item())
            rec_losses.append(loss_rec.item())
            reg_losses.append(loss_reg.item())
            
    #         print(f"Batch loss: {batch_loss}")
            optimizer.zero_grad()
            batch_loss.backward()

    #        torch.nn.utils.clip_grad_norm_(train_model.parameters(), 4)
            optimizer.step()
        
        batch_train_loss = np.mean(train_losses)
        batch_rec_loss = np.mean(rec_losses)
        batch_reg_loss = np.mean(reg_losses)
        
        total_train_losses.append([ep, batch_train_loss])
        total_rec_losses.append([ep, batch_rec_loss])
        total_reg_losses.append([ep, batch_reg_loss])
        scheduler.step(batch_train_loss)
        
        # Evaluation
        train_model.eval()
        with torch.no_grad():
            user_emb, item_emb, gcnEmbedsLst, hyperEmbedsLst = train_model()
            measure, performance_ep = rec.fast_evaluation(ep, user_emb, item_emb)
            for key, value in performance_ep.items():
                if key in performance_dict:
                    performance_dict[key].append(value)
                else:
                    continue
          
    rec.save_model(train_model, hyper_params)
    rec.save_loss(total_train_losses, total_rec_losses, total_reg_losses, hyper_params)
    rec.save_perfomance_training(performance_dict, hyper_params)
    
    
#         df_train = df_train.append(new_data, ignore_index=True)
#     current_time = datetime.now()
#     df_train.to_csv('results/'+ model + '/' + model+'@' + idx + '_train_['+ params + ']'+ str(current_time) +'.csv', index=False)
    
    return user_emb, item_emb

## Test

In [10]:
def test(rec, user_emb, item_emb, hyper_params):
    
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
        sys.stdout.write(r)
        sys.stdout.flush()

    # predict
    rec_list = {}
    user_count = len(rec.data.test_set)
    for i, user in enumerate(rec.data.test_set):
        # s_find_candidates = time.time()
        candidates = predict(user, rec, user_emb, item_emb)
        # e_find_candidates = time.time()
        # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
        # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
        rated_list, li = rec.data.user_rated(user)
        for item in rated_list:
            candidates[rec.data.item[item]] = -10e8

        # s_find_k_largest = time.time()
        ids, scores = find_k_largest(rec.max_N, candidates)
        # e_find_k_largest = time.time()
        # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
        item_names = [rec.data.id2item[iid] for iid in ids]
        rec_list[user] = list(zip(item_names, scores))
        if i % 1000 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    result = ranking_evaluation(rec.data.test_set, rec_list, rec.topN)
    print(result)
    rec.save_performance_test(result, hyper_params)
    
#     df_test = pd.DataFrame(new_data, index=[0])
#     current_time = datetime.now()
#     path = 'results/' + model + '/' + model + ' ' + str(current_time)
#     file_path = path + '/' + model+'@' + str(idx) + '_test_'+ str(params) + ''+ str(current_time) +'.csv'
#     df_test.to_csv(file_path, index=False)
     
    

## Main

In [11]:
model = 'HCCF'
if model not in ['HGNN', 'LightGCN', 'HCCF', 'DHCF']:
    print("No model found.")
config = ModelConf('./conf/' + model + '.conf')

dataset = 'lastfm'
batchSize = int(config['batch_size'])
# lRate = float(config['learnRate'])
# maxEpoch = int(config['num.max.epoch'])
# reg = float(config['reg.lambda'])
maxEpoch = 500
lRates = [0.01, 0.001, 0.0001]
lrDecays = [0.9, 0.8, 0.5]
regs = [0.01, 0.001, 0.0001]
hyperDims = [32, 64, 128]
inputDims = [8, 16, 32]
ps = [0.1, 0.4, 0.6]
dropRates = [0.1, 0.3, 0.5]
nLayers = [1, 2, 3]
hyperedge_num = [32, 64, 128]

if dataset == 'lastfm':
    training_set = "./dataset/music/train.txt"
    test_set = './dataset/music/test.txt'
elif dataset == 'ml-1m':
    training_set = './dataset/ml-1m/train.txt'
    test_set = './dataset/ml-1m/test.txt'
    
hyperparameters = [lRates, lrDecays, regs, hyperDims, inputDims, ps, dropRates, nLayers, hyperedge_num, dataset]

In [12]:
reg = 0.01
dataset = 'lastfm'
model = 'HCCF'
args = {
    'p':  0.1,
    'drop_rate': 0.5,
    'n_layers': 2,
    'input_dim': 32,
    'hyper_dim': 128,
    'hyperedge_num': 128,
    'lr': 0.001,
    'reg': reg,
    'leaky': 0.5,
    'dataset': dataset
}
training_data = FileIO.load_data_set('./dataset/' + dataset + '/' +config['training.set'], config['model.type'])
test_data = FileIO.load_data_set('./dataset/' + dataset + '/'  +config['test.set'], config['model.type'])
knowledge_set =  ''
rec = GraphRecommender(config, training_data, test_data, knowledge_set, **args)
if model == 'HGNN':
    train_model = Model(rec.config, rec.data, args).to(device)
elif model == 'LightGCN':
    train_model = LGCN_Encoder(rec.data, args['input_dim'], args['n_layers'])
elif model == 'HCCF':
    train_model = HCCFEncoder(rec.data, args['input_dim'], args['n_layers'], args['hyperedge_num'], args['leaky'], args['drop_rate'])

optimizer  = torch.optim.Adam(train_model.parameters(), lr=args['lr'], weight_decay=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=5)
user_emb, item_emb = train(args)
test(rec, user_emb, item_emb, args)

/home/aiotlab3/Workspace/hungvv/SELFRec/base/torch_interface.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  i = torch.LongTensor([coo.row, coo.col])


Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 3.072379 s
Measure time: 0.014938 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 1, Hit Ratio:0.02538  |  Precision:0.01563  |  Recall:0.02513  |  NDCG:0.02724
*Best Performance* 
Epoch:fast_evaluation 1, Hit Ratio:0.02538  |  Precision:0.01563  |  Recall:0.02513  |  NDCG:0.02724
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.162624 s
Measure time: 0.015055 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Per

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.072481 s
Measure time: 0.006314 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 248, Hit Ratio:0.22552  |  Precision:0.14033  |  Recall:0.22521  |  NDCG:0.2509
*Best Performance* 
Epoch:fast_evaluation 231, Hit Ratio:0.23774  |  Precision:0.14794  |  Recall:0.23811  |  NDCG:0.25912
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.073231 s
Measure time: 0.006441 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current 

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.072710 s
Measure time: 0.006117 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 261, Hit Ratio:0.238  |  Precision:0.1481  |  Recall:0.23887  |  NDCG:0.25844
*Best Performance* 
Epoch:fast_evaluation 258, Hit Ratio:0.2394  |  Precision:0.14897  |  Recall:0.23904  |  NDCG:0.26239
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.072621 s
Measure time: 0.006436 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Per

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.076751 s
Measure time: 0.006566 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 274, Hit Ratio:0.23711  |  Precision:0.14754  |  Recall:0.23914  |  NDCG:0.26506
*Best Performance* 
Epoch:fast_evaluation 268, Hit Ratio:0.24131  |  Precision:0.15016  |  Recall:0.24285  |  NDCG:0.26483
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.072726 s
Measure time: 0.006271 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current

...

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.072613 s
Measure time: 0.006484 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 443, Hit Ratio:0.25404  |  Precision:0.15808  |  Recall:0.25533  |  NDCG:0.28021
*Best Performance* 
Epoch:fast_evaluation 442, Hit Ratio:0.25519  |  Precision:0.1588  |  Recall:0.25671  |  NDCG:0.2804
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.073127 s
Measure time: 0.006574 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current P

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.072044 s
Measure time: 0.006260 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 456, Hit Ratio:0.25519  |  Precision:0.1588  |  Recall:0.25615  |  NDCG:0.28152
*Best Performance* 
Epoch:fast_evaluation 446, Hit Ratio:0.25583  |  Precision:0.15919  |  Recall:0.25776  |  NDCG:0.28314
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.073534 s
Measure time: 0.007253 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current 

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.072845 s
Measure time: 0.006466 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 469, Hit Ratio:0.25341  |  Precision:0.15769  |  Recall:0.25421  |  NDCG:0.28111
*Best Performance* 
Epoch:fast_evaluation 446, Hit Ratio:0.25583  |  Precision:0.15919  |  Recall:0.25776  |  NDCG:0.28314
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.072651 s
Measure time: 0.006234 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.072257 s
Measure time: 0.006139 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 482, Hit Ratio:0.25443  |  Precision:0.15832  |  Recall:0.2556  |  NDCG:0.2802
*Best Performance* 
Epoch:fast_evaluation 471, Hit Ratio:0.25672  |  Precision:0.15975  |  Recall:0.25863  |  NDCG:0.284
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.074716 s
Measure time: 0.006259 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Per

Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.084570 s
Measure time: 0.007196 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 495, Hit Ratio:0.25506  |  Precision:0.15872  |  Recall:0.25579  |  NDCG:0.28303
*Best Performance* 
Epoch:fast_evaluation 494, Hit Ratio:0.25774  |  Precision:0.16038  |  Recall:0.25946  |  NDCG:0.28278
------------------------------------------------------------------------------------------------------------------------
Evaluating the model...
Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 0.071817 s
Measure time: 0.006190 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current

In [13]:
# for idx, params in enumerate(product(*hyperparameters)):
#     lr, lr_decay, reg, hyper_dim, input_dim, prob, drop_rate, n_layers, hyperedge_num = params
#     print("hyperparameters:{}".format(params))
#     args = {
#         'p': prob,
#         'drop_rate': drop_rate,
#         'n_layers': n_layers,
#         'input_dim': input_dim,
#         'hyper_dim': hyper_dim,
#         'hyperedge_num': hyperedge_num,
#         'lr': lr,
#         'reg': reg,
#         'leaky': 0.5,
#         'dataset': dataset
#     }

#     training_data = FileIO.load_data_set(training_set, config['model.type'])
#     test_data = FileIO.load_data_set(test_set, config['model.type'])

#     rec = GraphRecommender(config, training_data, test_data, **args)
#     if model == 'HGNN':
#         train_model = Model(rec.config, rec.data, args).to(device)
#     elif model == 'LightGCN':
#         train_model = LGCN_Encoder(rec.data, input_dim, args['n_layers'])
#     elif model == 'HCCF':
#         train_model = HCCFEncoder(rec.data, input_dim, args['n_layers'], args['hyperedge_num'], args['leaky'], args['drop_rate'])

#     optimizer  = torch.optim.Adam(train_model.parameters(), lr=lr)
#     scheduler = ReduceLROnPlateau(optimizer, 'min', factor=lr_decay, patience=5)
#     user_emb, item_emb = train(args)
#     test(rec, user_emb, item_emb, args)